# Experiment: Lokale Ausführung von LLMS mit Langchain  
   
Zu Beginn des Experiments war es ungewiss, ob spezifische Abhängigkeiten zu bestimmten Python-Versionen bestehen. Deshalb entschied ich mich, mit Python-Umgebungen zu arbeiten.   
  
Für diesen Zweck habe ich Miniconda installiert, da ich bereits einige Erfahrungen mit dieser Umgebung gesammelt habe.   
  
Ich habe eine Umgebung mit der Version 3.11.8 erstellt, indem ich die folgenden Befehle ausgeführt habe:   
  
``` bash  
conda create -n langchain python=3.11.8   
conda activate langchain  
conda install langchain -c conda-forge  
```  
   
Informationsquellen:  
- [LLMS lokal ausführen](https://python.langchain.com/docs/guides/local_llms)

# Experiment: [Ollama](https://python.langchain.com/docs/guides/local_llms#ollama)  
   
Ollama ist ein Modul, das als Server fungiert, um Modelle für die Interaktion über die Shell bereitzustellen.    
Es ermöglicht das Herunterladen, Speichern und Ausführen von Modellen auf einer lokalen Maschine, was besonders nützlich ist, wenn Sie mit großen Modellen arbeiten oder eine stabile Netzwerkverbindung nicht garantiert ist.  
   
Zuerst muss das Ollama-Modul installiert werden. Sie können dies mit dem folgenden Befehl tun:  
``` bash  
conda install ollama  
```   
  
Nach der Installation kann der Ollama-Server mit dem folgenden Befehl gestartet werden:  
``` bash  
ollama serve  
```  
   
In einer anderen Shell kann nun das gewünschte Modell heruntergeladen werden. Der folgende Befehl lädt beispielsweise das Modell "llama2:7b" herunter:  
``` bash  
ollama pull llama2:7b  
```   
  
Solange der `ollama server` läuft, können Sie nun direkt über die Shell mit dem heruntergeladenen Modell interagieren.    
Der folgende Befehl startet die Interaktion mit dem Modell "llama2:7b":  
``` bash  
ollama run llama2:7b  
```

In [2]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2:7b")

llm.invoke("Wie viele Menschen gibt es auf der Welt?")

'\nLaut den United Nations Department of Economic and Social Affairs gibt es aktuell (Stand: Juli 2022) etwa 7,9 billion people (7.9 × 10^9) auf der Welt.\n\nDie genaue Anzahl der Menschen auf der Welt kann jedoch schwanken, da die Bevölkerung ständig durch Geburten, Sterbeiten und Migration verändert wird. Zusätzlich gibt es immer wieder Unbestimmtheiten in der Datenbasis von Volkszählungen und anderen Quellen, die die genaue Anzahl der Menschen auf der Welt festlegen müssen.'

# Experiment: [Llama.cpp](https://python.langchain.com/docs/guides/local_llms#llama.cpp)  
   
Llama.cpp ist ein C++-Implementierung des Llama-Modells.   
Es ermöglicht eine effiziente Ausführung von Llama-Modellen auf lokalen Maschinen, indem es die Hardware-Ressourcen, einschließlich CUDA-fähiger Grafikkarten, optimal nutzt.  
   
Zunächst muss das `llama-cpp-python` Modul installiert werden. Dieses Modul stellt die Python-Schnittstelle für die C++-Implementierung von Llama bereit.    
Da ich eine Nvidia-Grafikkarte mit CUDA-Unterstützung besitze, erfolgt die Installation mit CUDA-Unterstützung wie folgt:  
   
``` bash  
CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1   
pip install llama-cpp-python  
```  
   
Nach der Installation von `llama-cpp-python` sollte ein Modell heruntergeladen werden. 
Eine zuverlässige Quelle für passende Modelle ist Hugging Face:     
https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/blob/main/llama-2-7b-chat.Q4_K_S.gguf  
   
Quellen:  
- [Langchain Anleitung](https://python.langchain.com/docs/integrations/llms/llamacpp)

In [6]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp
from pathlib import Path

# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path=f"{Path.home()}/Models/llama-2-7b-chat.Q4_K_S.gguf",
    temperature=0.75,
    max_tokens=2000,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

prompt = """
Wie viele Menschen gibt es auf der Welt?
"""
llm.invoke(prompt)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /home/kosta/Models/llama-2-7b-chat.Q4_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count 


Die genaue Zahl der Menschen, die auf der Welt leben, ist schwierig zu bestimmen, da sie sich ständig ändert und von verschiedenen Faktoren abhängt. Die United Nations Population Division (UNPD) gibt eine Schätzung der weltweiten Bevölkerung für 2022 an, die jedoch immer nur eine grobe Schätzung ist.
According to the UNPD, there were approximately 7.9 billion people in the world as of 2022. This number is based on estimates and projections, and it is important to note that the actual number of people on Earth can vary depending on factors such as birth rates, death rates, and migration patterns.
It is worth noting that the population of the world is growing rapidly, with an estimated rate of 1.09% annually from 2019 to 2050. This means that the world's population is expected to reach 8.5 billion by 20

KeyboardInterrupt: 

# Experiment: [GPT4All](https://python.langchain.com/docs/guides/local_llms#gpt4all)  
   
GPT4All ist eine Plattform, die Zugriff auf eine Reihe von Modellen für die Generierung von Text mittels Künstlicher Intelligenz bietet.    
Es unterstützt eine Vielzahl von Modellen, einschließlich solcher, die auf der GPT-Architektur basieren. Diese Modelle können lokal auf Ihrer Maschine ausgeführt werden, was insbesondere bei begrenzter oder instabiler Internetverbindung nützlich ist.  
   
Zunächst muss eines der [unterstützten Modelle](https://gpt4all.io/index.html) heruntergeladen werden.   
Zum Beispiel kann das Modell "mistral-7b-openorca.Q4_0.gguf" mit dem folgenden Befehl heruntergeladen werden:  
   
``` bash  
mkdir Models  
wget https://gpt4all.io/models/gguf/mistral-7b-openorca.Q4_0.gguf -O Models/mistral-7b-openorca.Q4_0.gguf  
```   
  
Quelle:  
- [GPT4All Anleitung](https://python.langchain.com/docs/integrations/providers/gpt4all#gpt4all-1): Diese Anleitung zeigt, wie man GPT4All in Langchain integriert.

In [8]:
from langchain_community.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from pathlib import Path

# There are many CallbackHandlers supported, such as
# from langchain.callbacks.streamlit import StreamlitCallbackHandler

callbacks = [StreamingStdOutCallbackHandler()]
model = GPT4All(model=f"{Path.home()}/Models/mistral-7b-openorca.Q4_0.gguf", n_threads=8)

# Generate text. Tokens are streamed through the callback manager.
model("Once upon a time, ", callbacks=callbacks)

10 years ago, I was in the process of moving to New York City. It was an exciting and scary time for me as it meant leaving my family and friends behind to start fresh in this big city.


'10 years ago, I was in the process of moving to New York City. It was an exciting and scary time for me as it meant leaving my family and friends behind to start fresh in this big city.\n'

# Experiment: [llamafile](https://python.langchain.com/docs/guides/local_llms#llamafile)  
   
Ein Llamafile ist eine spezielle Datei, die ein Llama-Modell und den Code, der zum Ausführen des Modells benötigt wird, in einer einzigen ausführbaren Datei kapselt.    
Dies macht es einfach, Llama-Modelle auf verschiedenen Maschinen auszuführen, ohne dass zusätzliche Software oder Bibliotheken benötigt werden.  
   
Die Llamafiles können direkt im Servermodus ausgeführt werden. Hier ist ein Beispiel, wie Sie ein Llamafile herunterladen und ausführen können:  
   
``` bash  
wget https://huggingface.co/jartine/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile -O Models/TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile  
   
chmod +x Models/TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile  # run if you're on MacOS, Linux, or BSD  
./Models/TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile --server --nobrowser  
```  
   
Mit dem Befehl `chmod +x` wird die Datei ausführbar gemacht (notwendig für MacOS, Linux oder BSD). Der Befehl `--server` startet das Modell im Servermodus, und `--nobrowser` verhindert, dass ein Webbrowser automatisch geöffnet wird.

```
Quellen:
- https://huggingface.co/models?other=llamafile


In [ ]:
from langchain_community.llms.llamafile import Llamafile

llm = Llamafile()

llm.invoke("Tell me a joke")

" and I'll tell you a story.\nFor one, he is the tallest man in the world.\nAnd three, he eats with his mouth open.\nSure, here's another joke: How does a lion feel about being a zoo animal?\nAnswer: It's like being stuck in traffic!\nI like my eggs over easy.\nI am tired of your lies.\nWhat's the difference between a tree and an elephant?\nA tree can't talk, but an elephant does."